<a href="https://colab.research.google.com/github/nazike/dataoptions/blob/main/OptionsAnalysisPricing191021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
#read file from the github repo
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
df = pd.read_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/DataIn/V2.csv')
df.head()

,82.8111637168575,0.842681430911326,0.031938881239426,0.0456901041648581,0.17690706531888,18.4094117640506,18.4210377107872
0,113.935171,0.285741,0.008467,0.023575,0.538127,6.612110,6.612107
1,98.305405,0.813916,0.019547,0.018027,0.048863,2.601042,2.626842
2,109.189731,0.713558,0.022951,0.015280,0.303538,6.397872,6.430745
3,83.402084,2.879962,0.042272,0.058719,0.163288,20.609465,20.903553
4,127.549831,0.907992,0.026664,0.017533,0.562713,12.093558,12.156191


In [19]:
#merge two dataframes
df.columns = columns

In [20]:
df.describe()

,asset_price,maturity,rate,div,ivol,eu_price,am_price
count,100455.000000,100455.000000,1.004550e+05,1.004550e+05,100455.000000,100455.000000,100455.000000
mean,98.141144,1.214371,3.479940e-02,3.525762e-02,0.339526,15.501890,15.961782
std,16.859969,1.256565,2.450852e-02,2.471843e-02,0.156516,10.418466,10.663400
min,70.000660,0.100010,5.427117e-07,5.427117e-07,0.000015,0.000000,0.500121
25%,83.673626,0.423637,1.555361e-02,1.585122e-02,0.213415,6.687584,6.905906
50%,97.373486,0.725755,3.107879e-02,3.154336e-02,0.345592,14.226921,14.696291
75%,112.142653,1.246729,4.666650e-02,4.707699e-02,0.473691,23.108803,23.921224
max,129.999578,4.999891,9.999842e-02,9.999347e-02,0.599988,63.146201,63.146473


In [24]:
# Read the data
X_full = df.copy()
columns = ['asset_price', 'maturity', 'rate', 'div', 'ivol', 'eu_price', 'am_price'] 
X_full.columns = columns
X_full['target'] = (X_full['am_price']/X_full['eu_price']-1).copy()

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['target'], inplace=True)
y = X_full.target
y_amer = X_full['am_price']
X_full.drop(['am_price'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/v2fulldata.csv', index=False)
X_full.drop(['target'], axis=1, inplace=True)
X_full.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/v2fulldatatrain.csv', index=False)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality 
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()



In [22]:
len(X_train)

80364

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
#model = XGBRegressor(n_estimators=5000, random_state=0, learning_rate=0.07, n_jobs=4)

model = XGBRegressor(max_depth=6, reg_alpha=0.0002433756223211188, colsample_bytree=0.8242913620513932, learning_rate=0.05640249301683806, gamma=4.5885092732685184e-05,
                        reg_lambda=0.014400178832201075, min_child_weight=0.003963394614607396, 
                        n_estimators=10000, n_jobs=4, random_state=0, subsample=0.8586706877598019)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)



XGBoostError: [18:20:27] ../src/data/data.cc:367: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001336dea54 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x0000000133728ff4 xgboost::MetaInfo::SetInfo(char const*, void const*, xgboost::DataType, unsigned long) + 324
  [bt] (2) 3   libxgboost.dylib                    0x00000001336e0c74 XGDMatrixSetDenseInfo + 164
  [bt] (3) 4   libffi.7.dylib                      0x000000010bf49ead ffi_call_unix64 + 85
  [bt] (4) 5   ???                                 0x0000000307332ba0 0x0 + 13005695904



In [9]:
# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MSE',mean_squared_error(y_valid, preds))

print('MAE:', mean_absolute_error(y_valid, preds))

MSE 0.008484357431398526
MAE: 0.03898596273268801


In [10]:
preds_full=clf.predict(X_full)
preds_full

array([-2.5773565e-03,  3.1377017e-02,  3.1383466e-02, ...,
        3.2131438e+00, -3.0103406e-02,  1.2244771e-02], dtype=float32)

In [16]:
#create a dataframe with the predicted values and write it to csv   
predicted_df=df.copy()
predicted_df['predicted_target']=np.maximum(np.zeros(len(preds_full)),preds_full)
predicted_df['predicted_am_price']=predicted_df['predicted_target']+predicted_df['eu_price']
predicted_df.to_csv('/Users/nazymazimbayev/Documents/git-projects/dataoptions/full_predicted_dataoptions.csv', index=False)

In [ ]:
#calculation of the neural net time 
X_sample = X_full.iloc[0:1].copy()


import timeit
def test_nn():
    return clf.predict(X_sample)

loop = 1000

result_nn = timeit.timeit('test_nn()', globals=globals(), number=loop)
print(result_nn, 'seconds per loop times of test function')
print(result_nn/loop, 'seconds per each clf.predict()')

3.019512332999966 seconds per loop times of test function
0.003019512332999966 seconds per each clf.predict()


In [ ]:
#function to calculate binomial option price

import numpy as np

def binomial_put(S, K, T, R, div, sig, n):
    
    h = T/n
    Rinv = np.exp(-R*h)
    a = np.exp((R-div)*h)
    b2 = (a**2)*(np.exp(h*sig**2)-1)
    tmp = a**2 + b2 + 1
    u = (tmp + (tmp**2 - 4*a**2)**0.5)/(2*a)
    d = 1/u
    p = (a-d)/(u-d)
    q = 1-p
    pp = Rinv*p
    qq = Rinv*q
    x = 2*n + 1
    
    s = np.zeros(x)
    s[n] = S
    
    for j in range(1,n+1):
        s[n+j] = s[n-1+j]*u
        s[n-j] = s[n+1-j]*d
        
    v = np.zeros(x)
    
    for j in range(1,x+2,2):
        v[j-1] = max(K-s[j-1],0)
        
    for i in range(n-1,0,-1):
        for j in range(-i,i+2,2):
            j1 = j+n+1
            v[j1-1] = max(K-s[j1-1],pp*v[j1]+qq*v[j1-2])
    
    y=pp*v[n+1]+qq*v[n-1]
    
    return y

In [ ]:
#parameters for the binomial put option price calcualtion , similar to X_full.iloc[0:1]

S = 91.063192
K = 100.0
T = 0.64276
R = 0.073981	
div = 0.069325	
sig = 0.312864
n=250


result = binomial_put(S, K, T, R, div, sig, n)


In [ ]:
#binomial option price calcualtion time
def test_bt():
    return binomial_put(S, K, T, R, div, sig, n)

loop = 1000

result_bt = timeit.timeit('test_bt()', globals=globals(), number=loop)
print(result_bt, 'seconds per loop times of test function')
print(result_bt/loop, 'seconds per each binomial_put')

21.317060082999888 seconds per loop times of test function
0.021317060082999887 seconds per each binomial_put


In [ ]:
#result, which shows how much faster the neural net is than the binomial tree
print(result_bt/result_nn, 'times nn faster than bt')

NameError: name 'result_bt' is not defined